# Hydroelectric Power Generation

In [ ]:
import numpy as np
import cvxpy as cvx

## Part 1

In [ ]:
K = 3
L = 2
J = 3
w_l = np.array([20, 15])
lam = np.array([10, 30, 8])
rho = np.array([30./47, 43./58, 50./72])
V_0 = np.array([120, 130])
V_min = np.zeros((L, K))
V_max = np.array([[95000, 95000, 95000], [11000, 11000, 11000]])
A = np.array([[15, 15, 15], [12, 12, 12]])
T_max = np.array([[55, 55, 55], [65, 65, 65], [80, 80, 80]])
T_min = np.zeros((J, K))

In [ ]:
def solve(K, L, J, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max):
    """Solve the optimization problem described in part 1 using a solver of your choice.
    
    Args:
        K (int): number of timestesp
        L (int): number of reservoirs
        J (int): number of turbines
        etc.
    
    Returns:
        the optimal water volumes, T_opt
        the optimal objective value, obj_val
    """
    # TODO: your implementation here
    T_opt = none
    obj_val = None
    return T_opt, obj_val

In [ ]:
# Solve the optimization problem
T_opt, obj_val = solve(K, L, J, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max)

# Round values for readability (optional)
obj_val = np.around(obj_val, decimals=7)
T_opt = np.around(T_opt, decimals=0)

# Print results
print("Optimal value:\n{}\n".format(obj_val))
print("Optimal water volumes, T_opt:\n{}".format(T_opt))

## Part 2 (Don't forget to re-initialize variables!)

In [ ]:
K = 3
L = 2
J = 3
w_l = np.array([20, 15])
lam = np.array([12, 30, 4])
rho = np.array([30./47, 43./58, 50./72])
V_0 = np.array([120, 130])
V_min = np.zeros((L, K))
V_max = np.array([[95000, 95000, 95000], [11000, 11000, 11000]])
A = np.array([[15, 15, 15], [12, 12, 12]])
T_min = np.zeros((J, K))
T_max = np.array([[55, 55, 55], [65, 65, 65], [80, 80, 80]])

In [ ]:
def solve_regularized(K, L, J, T_hat, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max, gamma):
    """Solve the optimization problem described in part 2 using a solver of your choice.
    
    Args:
        T_hat (numpy.ndarray): optimal solution T_opt from part 1
        gamma (float): regularization multiplier
        (remaining variables have same description as in part 1)
    
    Returns:
        the optimal water volumes, T_opt_reg
        the optimal objective value, obj_val_reg
    """
    # TODO: your implementation here
    T_opt_reg = None
    obj_val_reg = None
    return T_opt_reg, obj_val_reg

In [ ]:
# Solve the optimization problem
T_opt_reg, obj_val_reg = solve_regularized(K, L, J, T_opt, w_l, lam, rho, V_0, V_min, V_max, A, T_min, T_max, gamma=np.power(10, 10, dtype=np.float32))

# Round values for readability (optional)
obj_val_reg = np.around(obj_val_reg, decimals=7)
T_opt_reg = np.around(T_opt_reg, decimals=0)

# Print results
print("Optimal value:\n{}\n".format(obj_val_reg))
print("Optimal water volumes, T_opt_reg:\n{}".format(T_opt_reg))